In [1]:
import os
import json
import torch
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import shutil
import torchvision
#import mediapipe as mp
import ultralytics
import yaml
from tqdm import tqdm
from PIL import Image
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
data_root = "D:\\project\\data\\src\\humal_fall"
train_file_path = glob.glob(f"{data_root}\\train\\images")
train_label_root = f'{data_root}\\train\\labels'


In [6]:
# json 파일에 있는 bbox 정보를 yolo 형식으로
# class_name x_center y_center w h

def cvt_bbox_yolo(img_w, img_h, bbox):
    x, y, w, h = map(float, bbox.split(', '))
    
    x_center = (x + w/2) / img_w
    y_center = (y + h/2) / img_h
    
    width = w / img_w
    height = h / img_h
    
    return x_center, y_center, width, height

def cvt_json_yolo(json_data):
    img_w, img_h = map(int, json_data['metadata']['scene_res'].split(' x '))
    bbox = json_data['bboxdata']['bbox_location']
    x_center, y_center, width, height = cvt_bbox_yolo(img_w, img_h, bbox)  
    # class_id : 나중에 훈련 끝나고 정의
    class_id = ['1']   
    yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"    
    return yolo_format

def json_files_out(input_dir, output_dir):
    output_dir = f'{train_label_root}'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            json_path = os.path.join(input_dir, filename)
            
            # json read
            with open(json_path, 'r', encoding = 'utf-8') as f:
                json_data = json.load(f)
            
            # YOLO 형식으로 convert
            yolo_format = cvt_json_yolo(json_data)
            
            # 파일로 저장
            output_filename = json_data['metadata']['file_name'].replace('.JPG', '.txt')
            output_path = os.path.join(output_dir, output_filename)
            
            with open(output_path, 'w') as f:
                f.write(yolo_format)
            
            print('완료')

input_dir = train_file_path  
output_dir = data_root  

json_files_out(input_dir, output_dir)

TypeError: listdir: path should be string, bytes, os.PathLike or None, not list